In [1]:
import pickle
import pandas as pd

In [2]:
import os

In [3]:
%cd '/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/'

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


#### Remove amsetrdam questions from training set

In [4]:
with open ('data/results_ranking/with-nonfactual/questions_preprocessed.pickle', 'rb') as f:
    preprocessed_questions = pickle.load(f)

In [5]:
questions_for_ranking = pd.read_csv('data/question_answer/amsterdam_questions.csv')

In [6]:
good_samples = []
for sample in preprocessed_questions:
    if sample['Question'] in list(questions_for_ranking['Question']):
        good_samples.append(sample)

In [7]:
len(preprocessed_questions)

15557

In [8]:
len(good_samples)

70

In [9]:
train_dataset = []
for sample in preprocessed_questions:
    if not sample['Question'] in list(questions_for_ranking['Question']):
        train_dataset.append(sample)

In [10]:
len(train_dataset)

15487

In [11]:
from datasets import Dataset
df = pd.DataFrame(train_dataset)
train_dataset = Dataset.from_pandas(df)

In [12]:
train_dataset

Dataset({
    features: ['Year', 'Month', 'Question', 'Answer', 'Document', 'URLs', 'Preprocessed_Question', 'Preprocessed_Answer', 'question_id', '__index_level_0__'],
    num_rows: 15487
})

#### Load passages preprocessed

In [13]:
with open('data/results_ranking/passages_preprocessed.pickle', 'rb') as f:
    passages_preprocessed = pickle.load(f)

In [14]:
type(passages_preprocessed)

datasets.arrow_dataset.Dataset

#### Ranking

In [15]:
from src_clean.ranking.sparse_ranking import perform_tfidf_search, perform_random_search
from src_clean.ranking.evaluation import calculate_metrics_answer_similarity, simulate_answer

#### TFIDF

In [16]:
results_tfidf = perform_tfidf_search(train_dataset, passages_preprocessed, k=10) # 6 mins for whole collection

In [17]:
len(results_tfidf)

15487

In [18]:
with open ('data/results_ranking/with-nonfactual/training_sample_tfidf.pickle', 'wb') as f:
    pickle.dump(results_tfidf,f)